# Context
This notebook drives the training process for different models.

In [1]:
# Set project's environment variables
import os
import sys
from dotenv import load_dotenv
load_dotenv(dotenv_path="../project.env")
sys.path.append(os.environ["PYTHONPATH"])

import pickle

In [2]:
# Import project-wide and PH2 specific variables and functions
import superheader as sup
import TRAINheader as train



Chosen class grouping: two-classes


Directory /Users/diego/Desktop/iteso/TOG/ exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/src exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/bin exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/media exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/scores exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data/PH3/two-classes exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data/PH1/two-classes exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data/PH2/two-classes exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data/PH3/two-classes exists. Continuing with execution
device: mps

In [3]:
TRAIN_classes = 'two-classes'

# Model training

## KNN

### Active Hand prediction

In [4]:
train.arch.find_best(sup.DATA_AH_PF, sup.active_hand_col, TRAIN_classes, sup.TRAIN_KNN_CODE)

updating best... 0.8892215568862275
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'AHpf', 'label_col': 'active_hand', 'class_list': 'two-classes'}
	{'arch': 'KNN', 'k': 1}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/AHpf exists. Continuing with execution
updating best... 0.9021956087824351
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'AHpf', 'label_col': 'active_hand', 'class_list': 'two-classes'}
	{'arch': 'KNN', 'k': 3}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/AHpf exists. Continuing with execution
updating best... 0.907185628742515
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'AHpf', 'label_col': 'active_hand', 'class_list': 'two-classes'}
	{'arch': 'KNN', 'k': 5}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/AHpf exists. Continuing with execution
updating best... 0.908183632734531
	{'PH2': True, 

### Sign prediction

In [5]:
train.arch.find_best(sup.DATA_S_PF, sup.class_numeric_column, TRAIN_classes, sup.TRAIN_KNN_CODE)

updating best... 0.54296875
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'Spf', 'label_col': 'class_numeric', 'class_list': 'two-classes'}
	{'arch': 'KNN', 'k': 1}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/Spf exists. Continuing with execution
updating best... 0.55859375
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'Spf', 'label_col': 'class_numeric', 'class_list': 'two-classes'}
	{'arch': 'KNN', 'k': 2}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/Spf exists. Continuing with execution
updating best... 0.560546875
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'Spf', 'label_col': 'class_numeric', 'class_list': 'two-classes'}
	{'arch': 'KNN', 'k': 3}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/Spf exists. Continuing with execution
updating best... 0.572265625
	{'PH2': True, 'PH3': True, 'reducer': 'pca

In [6]:
train.arch.find_best(sup.DATA_S_PV, sup.class_numeric_column, TRAIN_classes, sup.TRAIN_KNN_CODE)

updating best... 0.6511627906976745
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'two-classes'}
	{'arch': 'KNN', 'k': 1}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/Spv exists. Continuing with execution
updating best... 0.6744186046511628
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'two-classes'}
	{'arch': 'KNN', 'k': 4}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/Spv exists. Continuing with execution
updating best... 0.6976744186046512
	{'PH2': True, 'PH3': True, 'reducer': 'kpca', 'kernel': 'poly', 'n': 1, 'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'two-classes'}
	{'arch': 'KNN', 'k': 11}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/KNN/Spv exists. Continuing with execution
updating best... 0.7209302325581395
	{'PH2'

## BERT

In [ ]:
train.arch.find_best(sup.DATA_AH_PF, sup.class_numeric_column, TRAIN_classes, sup.TRAIN_BERT_CODE)

updating best... 0.5049900199600799
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'AHpf', 'label_col': 'class_numeric', 'class_list': 'two-classes', 'batch_size': 2048, 'seq_len': 1, 'input_dim': 1}
	{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'prajjwal1/bert-tiny', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 2e-05, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 1000}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/two-classes/BERT/AHpf/prajjwal1/bert-tiny does not exist. Creating it and continuing with execution
updating best... 0.5998003992015968
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 2, 'data_unit': 'AHpf', 'label_col': 'class_numeric', 'class_list': 'two-classes', 'batch_size': 2048, 'seq_len': 1, 'input_dim': 2}
	{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'prajjwal1/bert-tiny', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 2e-05, 'loss_fn

In [ ]:
train.arch.find_best(sup.DATA_S_PF, sup.class_numeric_column, TRAIN_classes, sup.TRAIN_BERT_CODE)

In [ ]:
train.arch.find_best(sup.DATA_S_PV, sup.class_numeric_column, TRAIN_classes, sup.TRAIN_BERT_CODE)

# Score saving

In [ ]:
import pandas as pd
from datetime import datetime

## KNN

In [ ]:
knn_scores_df = pd.DataFrame(sup.knn_score_tracker, columns=sup.knn_scores_columns)

In [ ]:
knn_scores_df

In [ ]:
knn_scores_df.sort_values(by="accuracy", ascending=False).head(50)

In [ ]:
now = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
sup.create_dir_if_not_exists(os.path.join(sup.TRAIN_SCORES_ROOT, TRAIN_classes, sup.TRAIN_KNN_CODE))
knn_scores_df.to_csv(os.path.join(sup.TRAIN_SCORES_ROOT, TRAIN_classes, sup.TRAIN_KNN_CODE, f"{now}.csv"), index=False)

## BERT

In [ ]:
bert_scores_df = pd.DataFrame(sup.bert_score_tracker, columns=sup.bert_scores_columns)

In [ ]:
bert_scores_df

In [ ]:
bert_scores_df.sort_values(by="accuracy", ascending=False).head(50)

In [ ]:
now = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
sup.create_dir_if_not_exists(os.path.join(sup.TRAIN_SCORES_ROOT, TRAIN_classes, sup.TRAIN_BERT_CODE))
bert_scores_df.to_csv(os.path.join(sup.TRAIN_SCORES_ROOT, TRAIN_classes, sup.TRAIN_BERT_CODE, f"{now}.csv"), index=False)